In [3]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import os
from tqdm import tqdm_notebook as tqdm
import time
from lxml import etree
from sklearn.metrics import r2_score
from datetime import timedelta
import numpy as np

In [27]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def pretty_print_result(search_result, fields=None):
    if fields is None:
        fields = []
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')


def get_score(search_result):
    res = []
    for hit in search_result['hits']['hits']:
        res.append((hit["_id"], hit["_score"]))
    res.sort(key = lambda x: x[1], reverse = True)
    return res


class Index:
    def __init__(self, index, settings):
        self.index_name = index
        self.settings = settings
        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360}])
        if self.es.indices.exists(index=index):
            self.es.indices.delete(index=index)
        self.es.indices.create(index=index, body=settings)

    def es_actions_generator(self, path_to_docs):
        for doc_name in tqdm(os.listdir(path_to_docs)):
            with open(f"{path_to_docs}/{doc_name}", "r", encoding="utf-8") as inf:
                doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
                doc = json.load(inf)           
            yield create_es_action(self.index_name, doc_id, doc)


    def add_documents(self, path_to_docs):
        try:
            for ok, result in parallel_bulk(self.es, self.es_actions_generator(path_to_docs), queue_size=4, thread_count=4,
                                        chunk_size=1000):
                  if not ok:
                     print(result)
        except Exception as e: 
            print(e)

    def get_doc_by_id(self, doc_id):
        return self.es.get(index=self.index_name, id=doc_id)['_source']

    def search(self, query, *args):
        return self.es.search(index=self.index_name, body=query, size=20)
        # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20



In [28]:
settings_1 = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
}

In [29]:
settings = {
        'mappings': {
            'properties': {
                'text': {
                    'type': 'text',
                    'analyzer': 'russian_complex',
                    'search_analyzer': 'russian_complex'
                }
            }
            
        },
        "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "russian_snow", "english_snow"]
                },
                'russian_complex': {
                    'char_filter': [
                        'yont'
                    ],
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'lowercase',
                        'russian_snow'
                    ]
                }
            },
            'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е'
                    ]
                }
            },
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '[a-zA-Z_0-9\u0400-\u04FF]{2,}',
                    'group': 0
                },
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            },
            "filter" : {
                "russian_snow" : {
                    "type" : "snowball",
                    "language" : "Russian"
                },
                "english_snow" : {
                    "type" : "snowball",
                    "language" : "English"
                }
            }
        }
    }
}

In [30]:
index = Index("docs", settings_1)

In [31]:
start = time.time()
index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:03:38.548274


In [32]:
stem_index = Index('stem_docs', settings)

In [33]:
start = time.time()
stem_index.add_documents("res/json")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))

KeyboardInterrupt: 

In [ ]:
stem_index.get_doc_by_id('1000039')

In [34]:
class Query:
    def __init__(self, task_id, query, relevant_docs):
        self.task_id = task_id
        self.query = query
        self.relevant_docs = relevant_docs


In [35]:

def json_query(query):
    return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                }
            ]
        }
    }
}
        


In [36]:
def pagerank_query(query):
    return  {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                },
                {
                    'rank_feature': {
                        'field': 'pagerank',
                        'saturation': {
                            'pivot': 10
                        },
                        'boost': '5.0'
                    }
                }
            ]
        }
    }
}

In [37]:
class Metrics:
    def __init__(self, p, r, r_precision, map_score):
        self.r = r
        self.p = p
        self.r_precision = r_precision
        self.map_score = map_score
    
    def __str__(self):
        return f"r = {self.r}\np = {self.p}\nr_precision = {self.r_precision}\nMAP = {self.map_score}"
    
    __repr__ = __str__


class SearchQualityChecker:
    def __init__(self, queries, index):
        self.queries = queries
        self.index = index
        self.results = {}
        self.metrics = {}
        
    def get_results(self, get_query=json_query):
        r_precision_total = 0
        map_score_total = 0
        r_total = 0
        p_total = 0
        for q in tqdm(self.queries):
            res = self.index.search(get_query(q))
            print(q.task_id)
            pretty_print_result(res)
            scores = get_score(res)
            metric = Metrics(p=self.p(20, q, scores), r=self.r(20, q, scores), r_precision=self.r_precision(q, scores),
                            map_score=self.map_score(q, scores, 20))
            p_total += metric.p
            r_total += metric.r
            r_precision_total += metric.r_precision
            map_score_total += metric.map_score
            self.metrics[q.task_id] = metric
        Q = len(self.queries)
        print(Q)
        return Metrics(p=p_total / Q, r=r_total / Q, r_precision=r_precision_total / Q, map_score=map_score_total / Q)
    
    def r_precision(self, query, search_res_score):
        return self.r(len(query.relevant_docs), query, search_res_score)
    
    def map_score(self, query, search_res_score, n):
        m = 0
        for k in range(1, n):       
            m += self.p(k, query, search_res_score)
        R = len(query.relevant_docs)
        return m / n
    
    def p(self, k, query, search_res_score):
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / k
    
    def r(self, k, query, search_res_score):
        R = len(query.relevant_docs)
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / R if R != 0 else 0 if len(search_res_score) > 0 else 1



In [57]:
def find_diff_metrics(quality_checker, other_checker, k=20, comp=lambda x : x.map_score):
     res = []
     for task_id in quality_checker.metrics:
        metric = quality_checker.metrics[task_id]
        other_metric = other_checker.metrics[task_id]
        res.append((task_id, abs(comp(metric) - comp(other_metric)), metric, other_metric))
     res.sort(reverse=True, key=lambda x: x[1])
     return res

In [48]:
def get_relevance():
    res = {}
    xml_tree = etree.parse("data/or_relevant-minus_table.xml")
    root = xml_tree.getroot()
    for task in root.getchildren():
        relevant_docs = set()
        for document in task.getchildren():
            if document.get("relevance") == "vital":
                relevant_docs.add(document.get("id"))
        res[task.get("id")] = relevant_docs
    print(len(res))
    return res


def generate_queries_plain_texts():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in root.getchildren():
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), query_text.text, relevances[task.get("id")]))
                except Exception as e:
                    print(e)
    print(len(res))
    return res

In [49]:
#queries = generate_queries_plain_texts()
#print(queries)
quality_checker = SearchQualityChecker(queries, index)
plain_text_res = quality_checker.get_results()

arw49633
Total documents: 1911
Doc 993272, score is 13.157524
Doc 990390, score is 13.157524
Doc 359461, score is 13.157524
Doc 360857, score is 12.175998
Doc 114021, score is 9.994495
Doc 863522, score is 9.937239
Doc 863494, score is 9.902861
Doc 599962, score is 9.885215
Doc 969889, score is 9.791648
Doc 802136, score is 9.790344
Doc 77084, score is 9.769339
Doc 89548, score is 9.744069
Doc 1319849, score is 9.6186695
Doc 623630, score is 9.608261
Doc 971553, score is 9.583519
Doc 1283004, score is 9.556349
Doc 1427859, score is 9.48445
Doc 86909, score is 9.48445
Doc 77545, score is 9.461813
Doc 658433, score is 9.452979
arw49662
Total documents: 0
arw49674
Total documents: 10000
Doc 74265, score is 20.83848
Doc 876069, score is 19.587797
Doc 1115273, score is 17.429165
Doc 19995, score is 17.099628
Doc 19214, score is 17.099628
Doc 1509861, score is 16.007002
Doc 74270, score is 15.681253
Doc 681222, score is 15.483906
Doc 1360296, score is 15.483906
Doc 878912, score is 14.794117

arw49944
Total documents: 9626
Doc 7317, score is 23.408134
Doc 942354, score is 22.699839
Doc 362460, score is 21.800516
Doc 1369472, score is 21.049526
Doc 860912, score is 19.23954
Doc 378916, score is 18.96234
Doc 1173677, score is 18.649426
Doc 1339324, score is 18.490355
Doc 1177576, score is 18.223392
Doc 1296174, score is 18.18085
Doc 1208015, score is 17.757048
Doc 1271351, score is 17.16899
Doc 1480002, score is 17.083025
Doc 857813, score is 16.92191
Doc 1467276, score is 16.744686
Doc 466123, score is 16.445835
Doc 378992, score is 16.202778
Doc 1309622, score is 15.204734
Doc 646295, score is 15.13085
Doc 1339827, score is 14.97822
arw49950
Total documents: 10000
Doc 489340, score is 24.557968
Doc 866689, score is 24.529598
Doc 1299659, score is 24.529598
Doc 269146, score is 23.233757
Doc 494355, score is 23.154278
Doc 1012870, score is 22.91935
Doc 265146, score is 22.587479
Doc 554088, score is 22.587479
Doc 586432, score is 22.379158
Doc 583987, score is 22.379158
Doc 

Doc 869740, score is 11.258926
Doc 772518, score is 11.098099
Doc 1153112, score is 10.972708
Doc 1151542, score is 10.855631
Doc 681812, score is 10.855631
Doc 875807, score is 10.602444
Doc 406748, score is 10.602444
arw50334
Total documents: 100
Doc 1417446, score is 13.323986
Doc 528825, score is 13.281955
Doc 1361098, score is 13.175722
Doc 962463, score is 12.746555
Doc 1460362, score is 12.555131
Doc 1460353, score is 12.139685
Doc 1004001, score is 11.968523
Doc 870369, score is 11.086902
Doc 87177, score is 10.9626665
Doc 1510788, score is 10.7207
Doc 1028943, score is 10.662691
Doc 540587, score is 10.662691
Doc 699734, score is 10.662691
Doc 51541, score is 10.492373
Doc 50633, score is 10.492373
Doc 51986, score is 10.492373
Doc 52335, score is 10.492373
Doc 1379381, score is 10.492373
Doc 20003, score is 10.492373
Doc 30273, score is 10.4352255
arw50340
Total documents: 90
Doc 1033624, score is 15.955619
Doc 382878, score is 15.824734
Doc 1467906, score is 15.546444
Doc 70

arw50640
Total documents: 10000
Doc 659122, score is 27.914932
Doc 1467824, score is 27.227854
Doc 525099, score is 26.654903
Doc 1083510, score is 25.425962
Doc 562614, score is 25.298504
Doc 437130, score is 24.784813
Doc 347413, score is 24.429518
Doc 63744, score is 24.409668
Doc 1509245, score is 24.305544
Doc 1241125, score is 24.095848
Doc 327487, score is 24.046532
Doc 1172102, score is 24.009014
Doc 414661, score is 23.79013
Doc 794798, score is 23.757137
Doc 797577, score is 23.631844
Doc 1213094, score is 23.582073
Doc 350749, score is 23.388721
Doc 987417, score is 23.358229
Doc 5256, score is 23.320189
Doc 1177738, score is 23.28132
arw50642
Total documents: 10000
Doc 587618, score is 30.704493
Doc 499796, score is 28.821005
Doc 1309114, score is 28.792028
Doc 833891, score is 28.571018
Doc 87687, score is 27.684397
Doc 667405, score is 27.168718
Doc 37719, score is 26.969955
Doc 1077038, score is 26.903975
Doc 1271751, score is 26.788918
Doc 1319406, score is 26.676342
Do

arw50992
Total documents: 10000
Doc 1486127, score is 20.211136
Doc 1089808, score is 19.603209
Doc 334467, score is 16.129107
Doc 542998, score is 16.103674
Doc 801873, score is 16.103674
Doc 536864, score is 15.039522
Doc 1189188, score is 14.620038
Doc 1137755, score is 14.620038
Doc 801484, score is 13.89085
Doc 1290210, score is 13.591004
Doc 98479, score is 13.591004
Doc 272140, score is 9.298777
Doc 1445097, score is 6.468836
Doc 334662, score is 6.093822
Doc 1228033, score is 6.086164
Doc 106557, score is 6.0330353
Doc 639087, score is 6.0330353
Doc 1040615, score is 6.0123744
Doc 543380, score is 6.0123744
Doc 1212644, score is 6.010377
arw51002
Total documents: 1027
Doc 524586, score is 15.47694
Doc 814021, score is 11.81953
Doc 1101954, score is 11.6875
Doc 686382, score is 11.591975
Doc 956521, score is 11.481452
Doc 723829, score is 11.2277775
Doc 1273459, score is 11.111229
Doc 21863, score is 11.07958
Doc 914871, score is 10.96565
Doc 822495, score is 10.9004345
Doc 9148

...

arw57618
Total documents: 3956
Doc 791703, score is 27.35584
Doc 1000332, score is 8.45192
Doc 1411661, score is 8.45192
Doc 378775, score is 8.409225
Doc 1055180, score is 8.384908
Doc 1374855, score is 8.36358
Doc 525314, score is 8.351188
Doc 503424, score is 8.31775
Doc 984070, score is 8.295311
Doc 1298617, score is 8.279227
Doc 1462057, score is 8.279227
Doc 694264, score is 8.279227
Doc 984382, score is 8.279227
Doc 692716, score is 8.279227
Doc 1453247, score is 8.264429
Doc 1422083, score is 8.253533
Doc 437414, score is 8.2478485
Doc 723595, score is 8.231344
Doc 791701, score is 8.217637
Doc 1376865, score is 8.1996975
arw57620
Total documents: 10000
Doc 1462411, score is 18.638817
Doc 1297346, score is 18.202877
Doc 1467505, score is 17.966133
Doc 462612, score is 17.966133
Doc 1446671, score is 16.907497
Doc 357361, score is 16.715446
Doc 647918, score is 16.202995
Doc 570822, score is 16.048801
Doc 1123039, score is 15.081566
Doc 702965, score is 14.983066
Doc 335557, sco

arw57881
Total documents: 10000
Doc 575214, score is 20.189432
Doc 965044, score is 19.294333
Doc 585578, score is 15.903329
Doc 105132, score is 12.282671
Doc 1430591, score is 11.899367
Doc 347330, score is 11.838234
Doc 962880, score is 11.655708
Doc 799700, score is 9.450956
Doc 1158877, score is 9.32595
Doc 1243202, score is 9.041545
Doc 1407793, score is 8.761847
Doc 1144476, score is 8.07221
Doc 1311572, score is 7.240443
Doc 1094376, score is 7.235602
Doc 311924, score is 7.1314797
Doc 1162458, score is 7.101149
Doc 76199, score is 7.074702
Doc 865655, score is 7.0254793
Doc 660609, score is 7.004841
Doc 1190789, score is 7.004841
arw57932
Total documents: 10000
Doc 880639, score is 19.948145
Doc 986290, score is 19.63608
Doc 906465, score is 19.34823
Doc 395954, score is 19.078842
Doc 446652, score is 17.954287
Doc 486491, score is 17.615168
Doc 1086269, score is 17.168764
Doc 49459, score is 17.122644
Doc 1378167, score is 16.943407
Doc 827938, score is 16.86401
Doc 1479444, 

arw58309
Total documents: 10000
Doc 893805, score is 29.099545
Doc 893800, score is 28.63488
Doc 573076, score is 27.136295
Doc 1272980, score is 26.227573
Doc 14202, score is 25.629623
Doc 1347725, score is 25.349981
Doc 963247, score is 25.271112
Doc 1085068, score is 24.866455
Doc 1287074, score is 24.617167
Doc 1221750, score is 24.447718
Doc 65978, score is 24.185253
Doc 539628, score is 24.12385
Doc 899211, score is 24.114517
Doc 1109251, score is 23.7959
Doc 903306, score is 23.693695
Doc 1043979, score is 23.564892
Doc 1282616, score is 23.563654
Doc 773606, score is 23.215055
Doc 1157976, score is 23.201338
Doc 87671, score is 22.980484
arw58328
Total documents: 10000
Doc 1181049, score is 14.501312
Doc 1069025, score is 14.300163
Doc 774457, score is 13.869848
Doc 678850, score is 13.771218
Doc 964505, score is 13.689619
Doc 876651, score is 13.689619
Doc 1079497, score is 13.689619
Doc 44756, score is 13.499004
Doc 420852, score is 13.374075
Doc 1079506, score is 13.374075
D

arw58642
Total documents: 10000
Doc 1491107, score is 20.404606
Doc 1297031, score is 14.113967
Doc 261338, score is 13.839907
Doc 684577, score is 12.944818
Doc 1408446, score is 12.636912
Doc 801491, score is 12.566297
Doc 599684, score is 12.552715
Doc 543015, score is 12.500737
Doc 863196, score is 12.500737
Doc 282598, score is 12.494591
Doc 293718, score is 12.385689
Doc 63780, score is 12.352356
Doc 1285445, score is 12.337606
Doc 1095927, score is 12.331781
Doc 647203, score is 12.173464
Doc 1345705, score is 12.019936
Doc 80777, score is 11.947611
Doc 1431940, score is 11.947611
Doc 444782, score is 11.801967
Doc 1140736, score is 11.796758
arw58656
Total documents: 10000
Doc 1011797, score is 18.854252
Doc 888082, score is 18.648142
Doc 885267, score is 15.682592
Doc 720243, score is 15.160116
Doc 622524, score is 14.612402
Doc 1160367, score is 14.23526
Doc 1215054, score is 13.591521
Doc 1009296, score is 8.266839
Doc 1409610, score is 0.30556768
Doc 1077248, score is 0.304

arw59116
Total documents: 5176
Doc 528773, score is 33.121437
Doc 476884, score is 32.836548
Doc 853932, score is 32.345642
Doc 527818, score is 29.908985
Doc 8377, score is 29.198898
Doc 363156, score is 28.15946
Doc 7810, score is 28.13807
Doc 1421242, score is 27.748856
Doc 529918, score is 25.342451
Doc 1327859, score is 25.20743
Doc 120056, score is 24.127384
Doc 800512, score is 23.981722
Doc 1143981, score is 23.776539
Doc 646927, score is 23.53299
Doc 1461932, score is 23.53299
Doc 1179158, score is 22.676788
Doc 1327100, score is 22.53567
Doc 855154, score is 22.473766
Doc 1483779, score is 22.142532
Doc 1368721, score is 22.067005
arw59118
Total documents: 10000
Doc 1102707, score is 22.294468
Doc 1267950, score is 20.357536
Doc 256824, score is 18.739723
Doc 604411, score is 18.726316
Doc 1276050, score is 17.700895
Doc 1401043, score is 16.954275
Doc 257572, score is 16.81118
Doc 1004627, score is 16.290197
Doc 565785, score is 15.953986
Doc 565501, score is 15.930285
Doc 1

In [41]:
plain_text_res

r = 0.20970577689512557
p = 0.30219378427787935
r_precision = 0.1801915553804202
MAP = 0.344494963276825

In [50]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
black_list = ["°", "№", "©", "...", "//", "://", "</", "\">", "=\"", "=\'", "\r", "\n", "\t"]
stem = Mystem()

def lemmatize(text):
    words = nltk.word_tokenize(text.lower())
    tokens = []
    for word in words:
        tokens.extend(stem.lemmatize(word))
    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
              and token not in russian_stopwords and token not in english_stopwords \
              and token not in black_list \
              and token.find("\r") == -1 \
              and token.find("\n") == -1 \
              and token.find("\t") == -1 \
              and not (token.isdigit() and len(token) == 1)]
    return ' '.join(tokens)

def generate_queries_lemmas():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), lemmatize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res



In [51]:
queries_lemmas = generate_queries_lemmas()

547



547


In [ ]:
queries_lemmas[8].query

In [52]:
lemma_index = Index("lemma_docs", settings_1)

start = time.time()
lemma_index.add_documents("data/json_filtered_tokens_texts")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))


0:02:46.363651


In [53]:
lemma_quality_checker = SearchQualityChecker(queries_lemmas, lemma_index)
lemma_res = lemma_quality_checker.get_results()

arw49633
Total documents: 5080
Doc 914238, score is 13.799336
Doc 1393503, score is 13.701229
Doc 1407642, score is 13.604507
Doc 567839, score is 13.571222
Doc 1385740, score is 13.571222
Doc 1504166, score is 12.54204
Doc 1016244, score is 12.132923
Doc 993272, score is 11.921361
Doc 990390, score is 11.921361
Doc 359461, score is 11.921361
Doc 360857, score is 11.359218
Doc 495806, score is 10.947564
Doc 854005, score is 10.142758
Doc 691503, score is 9.414591
Doc 1340878, score is 9.019914
Doc 1319901, score is 7.9332757
Doc 35586, score is 7.8946466
Doc 114021, score is 7.8873105
Doc 599962, score is 7.8450575
Doc 863522, score is 7.8382044
arw49662
Total documents: 0
arw49674
Total documents: 10000
Doc 1503640, score is 22.287449
Doc 674640, score is 22.287449
Doc 876069, score is 21.912388
Doc 49202, score is 20.201603
Doc 575107, score is 19.590178
Doc 1109272, score is 19.590178
Doc 664883, score is 19.063917
Doc 74265, score is 18.712236
Doc 284872, score is 18.100986
Doc 131

arw49938
Total documents: 10000
Doc 1304391, score is 15.79537
Doc 1370851, score is 11.94859
Doc 593233, score is 11.929664
Doc 108087, score is 11.817169
Doc 417346, score is 11.62208
Doc 824462, score is 11.510288
Doc 1052266, score is 11.509953
Doc 657686, score is 11.491143
Doc 918846, score is 11.426479
Doc 541451, score is 11.406113
Doc 1304086, score is 11.33633
Doc 541404, score is 11.316824
Doc 1300065, score is 11.224773
Doc 362534, score is 11.203213
Doc 1178551, score is 11.174843
Doc 362535, score is 11.17198
Doc 1184140, score is 11.16831
Doc 970087, score is 11.16831
Doc 857427, score is 11.110881
Doc 694597, score is 11.052099
arw49944
Total documents: 10000
Doc 7317, score is 20.949793
Doc 942354, score is 20.436937
Doc 624744, score is 19.993721
Doc 1173677, score is 19.758795
Doc 1480002, score is 19.615366
Doc 362460, score is 19.473494
Doc 1339324, score is 19.33974
Doc 1309622, score is 19.226002
Doc 1369472, score is 19.144844
Doc 1296174, score is 19.084396
Doc

arw50231
Total documents: 8
Doc 1225583, score is 14.690051
Doc 774111, score is 14.279586
Doc 1285919, score is 14.153366
Doc 623607, score is 13.208949
Doc 44774, score is 8.222673
Doc 877305, score is 6.9237366
Doc 768002, score is 6.9237366
Doc 38757, score is 6.9237366
arw50276
Total documents: 10000
Doc 4053, score is 22.949055
Doc 675632, score is 19.641657
Doc 878847, score is 19.247711
Doc 771209, score is 19.166359
Doc 422593, score is 18.929113
Doc 26891, score is 18.830061
Doc 47141, score is 18.696009
Doc 649450, score is 18.668884
Doc 1449392, score is 18.485468
Doc 815863, score is 18.419012
Doc 417668, score is 18.243214
Doc 1365208, score is 18.068588
Doc 679077, score is 17.922443
Doc 988276, score is 17.8506
Doc 280888, score is 17.714481
Doc 968919, score is 17.714481
Doc 41888, score is 17.666801
Doc 1253795, score is 17.597158
Doc 474809, score is 17.428158
Doc 522355, score is 17.425522
arw50280
Total documents: 9877
Doc 858190, score is 19.791473
Doc 969597, sco

arw50632
Total documents: 10000
Doc 24817, score is 21.761023
Doc 485757, score is 21.694103
Doc 257703, score is 21.353956
Doc 1402977, score is 20.872845
Doc 375369, score is 20.75485
Doc 1300095, score is 20.236612
Doc 489207, score is 20.236612
Doc 1329866, score is 20.090866
Doc 258258, score is 19.309658
Doc 293657, score is 19.295757
Doc 565729, score is 19.29205
Doc 1437819, score is 19.105576
Doc 494515, score is 18.657513
Doc 513816, score is 18.608042
Doc 1189020, score is 18.546522
Doc 1436220, score is 18.482197
Doc 1518370, score is 18.437813
Doc 1301645, score is 18.40222
Doc 943357, score is 18.383879
Doc 822492, score is 18.367775
arw50640
Total documents: 10000
Doc 659122, score is 24.24823
Doc 522523, score is 24.189165
Doc 112621, score is 24.086151
Doc 624498, score is 23.89555
Doc 665356, score is 23.714025
Doc 1467824, score is 23.695723
Doc 350749, score is 23.679964
Doc 1083510, score is 23.577042
Doc 463550, score is 23.558205
Doc 1358897, score is 23.46224
Do

arw50992
Total documents: 10000
Doc 1486127, score is 19.565475
Doc 1089808, score is 19.132927
Doc 334467, score is 15.8987
Doc 801873, score is 14.812623
Doc 542998, score is 14.812623
Doc 536864, score is 14.433851
Doc 1137755, score is 13.6394005
Doc 1189188, score is 13.6394005
Doc 98479, score is 12.620554
Doc 1290210, score is 12.620554
Doc 801484, score is 12.551125
Doc 272140, score is 8.664051
Doc 1445097, score is 5.631472
Doc 1356545, score is 5.550957
Doc 543380, score is 5.5301948
Doc 1040615, score is 5.5301948
Doc 106557, score is 5.527626
Doc 639087, score is 5.527626
Doc 71973, score is 5.524686
Doc 1319632, score is 5.524686
arw51002
Total documents: 4237
Doc 1185833, score is 16.427021
Doc 1230185, score is 15.08635
Doc 1030052, score is 14.435724
Doc 669017, score is 12.603048
Doc 8599, score is 12.269979
Doc 524586, score is 11.966372
Doc 826408, score is 11.139467
Doc 446178, score is 11.13147
Doc 1238116, score is 11.117227
Doc 671795, score is 11.003828
Doc 109

...

arw57724
Total documents: 2792
Doc 1477991, score is 15.483676
Doc 967070, score is 15.035362
Doc 777035, score is 13.091854
Doc 406471, score is 12.325357
Doc 807096, score is 12.041454
Doc 894758, score is 11.507176
Doc 707390, score is 11.2631445
Doc 304930, score is 11.2631445
Doc 599248, score is 10.834952
Doc 1351926, score is 10.58208
Doc 1507844, score is 10.565641
Doc 466206, score is 10.499098
Doc 1228731, score is 10.335282
Doc 22621, score is 10.2893915
Doc 982586, score is 10.2893915
Doc 91774, score is 10.2893915
Doc 1179215, score is 10.211017
Doc 1008040, score is 10.170642
Doc 681290, score is 10.109232
Doc 891479, score is 9.971743
arw57730
Total documents: 10000
Doc 1174018, score is 17.245808
Doc 336222, score is 17.23945
Doc 288198, score is 16.525717
Doc 1489835, score is 16.346264
Doc 1138109, score is 16.075714
Doc 1060671, score is 15.757256
Doc 1138110, score is 15.6366825
Doc 898099, score is 15.520697
Doc 1113800, score is 15.520697
Doc 1059714, score is 15.

arw58008
Total documents: 10000
Doc 106734, score is 50.02405
Doc 1218649, score is 49.218796
Doc 1275021, score is 49.027893
Doc 399971, score is 49.012314
Doc 608213, score is 47.69914
Doc 636588, score is 47.055447
Doc 71886, score is 44.96726
Doc 698433, score is 44.280148
Doc 106019, score is 43.675537
Doc 1043267, score is 43.012436
Doc 1040807, score is 42.084705
Doc 1227273, score is 41.90063
Doc 841626, score is 41.11454
Doc 1261746, score is 41.06623
Doc 441721, score is 40.335087
Doc 839993, score is 40.138596
Doc 1401139, score is 40.023357
Doc 1466732, score is 39.94716
Doc 599482, score is 39.898407
Doc 713313, score is 39.697487
arw58015
Total documents: 8109
Doc 1510576, score is 15.233423
Doc 1043401, score is 14.550827
Doc 1261909, score is 14.284216
Doc 988682, score is 13.822187
Doc 109418, score is 13.503086
Doc 395807, score is 13.443358
Doc 845077, score is 13.443358
Doc 833277, score is 13.32873
Doc 833395, score is 13.283126
Doc 1480707, score is 13.283126
Doc 

arw58392
Total documents: 10000
Doc 842188, score is 16.175564
Doc 636641, score is 16.175564
Doc 1015717, score is 15.46916
Doc 556287, score is 15.44821
Doc 15574, score is 15.324477
Doc 494193, score is 15.2632885
Doc 933653, score is 15.254921
Doc 496012, score is 15.088171
Doc 301744, score is 15.067539
Doc 1147735, score is 14.893277
Doc 652888, score is 14.801023
Doc 40309, score is 14.567307
Doc 636921, score is 14.34304
Doc 439543, score is 14.34304
Doc 1280088, score is 14.2866
Doc 288535, score is 14.127954
Doc 986645, score is 14.058027
Doc 512814, score is 13.908368
Doc 999239, score is 13.805296
Doc 639955, score is 13.756645
arw58403
Total documents: 10000
Doc 555898, score is 16.700348
Doc 28674, score is 16.700348
Doc 1191441, score is 16.22716
Doc 37808, score is 15.7009945
Doc 608433, score is 15.668861
Doc 629042, score is 15.558431
Doc 1296191, score is 15.546085
Doc 1084744, score is 15.473769
Doc 1181581, score is 15.412655
Doc 1206719, score is 15.391674
Doc 982

arw58766
Total documents: 10000
Doc 833525, score is 15.141186
Doc 350611, score is 12.648222
Doc 350286, score is 12.648222
Doc 1500235, score is 12.354792
Doc 549746, score is 12.081139
Doc 1501284, score is 11.924647
Doc 1489452, score is 11.924647
Doc 386749, score is 11.849148
Doc 1500346, score is 11.663614
Doc 1307895, score is 11.609453
Doc 1472176, score is 11.609453
Doc 428326, score is 11.609453
Doc 29359, score is 11.600216
Doc 1218054, score is 11.478485
Doc 1307111, score is 11.399263
Doc 1186164, score is 11.380814
Doc 1307839, score is 11.369194
Doc 1472050, score is 11.282434
Doc 1287826, score is 11.21966
Doc 665602, score is 11.152847
arw58809
Total documents: 10000
Doc 269524, score is 15.53888
Doc 1455933, score is 14.422829
Doc 860339, score is 14.175802
Doc 417770, score is 14.122268
Doc 1284785, score is 14.122268
Doc 249810, score is 14.066691
Doc 418973, score is 14.066691
Doc 694658, score is 13.93699
Doc 676271, score is 13.519023
Doc 773859, score is 13.519

arw59134
Total documents: 2098
Doc 1118630, score is 20.464365
Doc 1495348, score is 20.4038
Doc 365768, score is 18.87925
Doc 852863, score is 18.025393
Doc 1264509, score is 18.021502
Doc 1495347, score is 16.646132
Doc 1115155, score is 16.099173
Doc 371132, score is 14.649252
Doc 1180875, score is 14.649252
Doc 1482378, score is 14.003223
Doc 294087, score is 13.989494
Doc 505232, score is 13.885551
Doc 46091, score is 13.769841
Doc 31534, score is 13.697264
Doc 1118627, score is 13.587566
Doc 1309070, score is 13.434001
Doc 943816, score is 13.046581
Doc 1512569, score is 12.633415
Doc 501967, score is 12.18092
Doc 55478, score is 12.18092
arw59172
Total documents: 2797
Doc 369692, score is 9.168687
Doc 277937, score is 8.970928
Doc 863925, score is 8.937612
Doc 83248, score is 8.84964
Doc 1382640, score is 8.800354
Doc 59200, score is 8.768355
Doc 1020646, score is 8.731795
Doc 1290989, score is 8.724941
Doc 1060014, score is 8.705168
Doc 602377, score is 8.682893
Doc 1456605, sc

In [54]:
lemma_res

r = 0.2535468230071016
p = 0.3577696526508227
r_precision = 0.21191923008395347
MAP = 0.38558780597371284

In [58]:
find_diff_metrics(quality_checker, lemma_quality_checker)

[('arw58304', 0.884148104862114, r = 0.05555555555555555
  p = 0.05
  r_precision = 0.05555555555555555
  MAP = 0.06322031619051743, r = 1.0
  p = 0.9
  r_precision = 1.0
  MAP = 0.9473684210526315),
 ('arw52186', 0.734162416745621, r = 0.02631578947368421
  p = 0.05
  r_precision = 0.02631578947368421
  MAP = 0.0359385701587714, r = 0.39473684210526316
  p = 0.75
  r_precision = 0.39473684210526316
  MAP = 0.7701009869043924),
 ('arw53079', 0.7096398559730602, r = 0.0625
  p = 0.2
  r_precision = 0.0625
  MAP = 0.20178999492079988, r = 0.265625
  p = 0.85
  r_precision = 0.265625
  MAP = 0.9114298508938601),
 ('arw55168', 0.6842088800002887, r = 0.08333333333333333
  p = 0.1
  r_precision = 0.08333333333333333
  MAP = 0.04060674404714652, r = 0.375
  p = 0.45
  r_precision = 0.375
  MAP = 0.7248156240474352),
 ('arw56114', 0.6739408044226155, r = 0.0
  p = 0.0
  r_precision = 0.0
  MAP = 0.0, r = 0.375
  p = 0.45
  r_precision = 0.375
  MAP = 0.6739408044226155),
 ('arw53029', 0.65659

In [ ]:
id_to_pagerank = {}
with open('res/pagerank.txt','r') as f:
    for line in f:
        docId, docURL, rank = line.split()
        id_to_pagerank[int(docId)] = float(rank)

In [ ]:
len(id_to_pagerank)


In [ ]:
 for doc_name in tqdm(os.listdir("data/json_filtered_tokens_texts")):
        with open(f"data/json_filtered_tokens_texts/{doc_name}", "r+", encoding="utf-8") as inf:
            doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
            doc = json.load(inf)
            try:
                doc["pagerank"] = id_to_pagerank.get(doc_id)
            except:
                pass
            inf.seek(0)        # <--- should reset file position to the beginning.
            json.dump(doc, inf, indent=4, ensure_ascii=False)
            inf.truncate()

In [ ]:
settings_with_pagerank = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "pagerank": {
                "type": "rank_feature"
            }
        }
    }
}

In [ ]:
pr_index = Index("pagerank_index", settings_with_pagerank)

In [ ]:
pr_index.add_documents("data/json_filtered_tokens_texts")

In [ ]:
pr_quality_checker = SearchQualityChecker(queries_lemmas, pr_index)
pr_res = pr_quality_checker.get_results(pagerank_query)

In [ ]:
pr_res

In [ ]:
pr_index.get_doc_by_id(1000039)

In [ ]:
settings_titles = {
        'mappings': {
            'properties': {
                'content': {
                    'type': 'text',
                },
                'title': {
                    'type': 'text'
                }
            }
            
        }
    }